In [2]:
# Interpretability_5_3-5(Solutions to Exercises 5.3, 5.4 and 5.5)
# Author: Matthew Dixon
# Version: 1.0 (08.09.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Dixon M.F. I. Halperin and P. Bilokon, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 
# Solution courtesy of Shengnan Li

In [10]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
import math
from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.regularizers import l1,l2
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score, train_test_split
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score, mean_absolute_error
import statsmodels.api as sm
import statsmodels.stats.diagnostic as tds
from statsmodels.api import add_constant
from scipy import stats


Using TensorFlow backend.


# Simple Data Generation Process (DGP)

$Y=X_1+X_2 + \epsilon, ~X_1, X_2, \epsilon \sim N(0,1)$

In [0]:
M = 5000 # Number of sample 
np.random.seed(7) # set the seed for reproducebility of results
X = np.zeros(shape=(M,2))
X[:int(M/2),0]= np.random.randn(int(M/2))
X[:int(M/2),1]= np.random.randn(int(M/2))

X[int(M/2):,0]= -X[:int(M/2),0] # zero bias of mean with antithetic sampling
X[int(M/2):,1]= -X[:int(M/2),1]


eps= np.random.randn(M)
Y= 1.0*X[:,0] + 1.0*X[:,1] + eps

In [0]:
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=10)

# Exercise 5.3. Compare with a FFW Neural Network with one hidden layer (tanh activated)

From running the code, it is evident that the activation function that should be used for interpretability is tanh.  When the number of hidden units is low, all three cases (no activation, tanh activation, and ReLU activation) provide fairly reasonable results, with means somewhat close to the true sensitivity values and small standard deviations.  
However, as the number of hidden units increases, tanh is the activation function that remains the best at interpreting the sensitivities.  Using tanh activation, the means get closer to the true values of the sensitivities, and the standard deviations remain small.  On the contrary, when ReLU activation is used, it can be seen that the sensitivity approximations are closer to the true values when the number of hidden units are small.  As the number of hidden units increases (above 100), the mean approximations do not capture the true sensitivities as well as tanh activation does.  As a result, tanh activation should be used over ReLU and no activation when it comes to capturing the sensitivities.

In [0]:
# number of hidden neurons
n=[10,50,100,200,500]

In [0]:
# Compute the sensitivities
# Assume that the activation function is tanh
def sensitivities_t(lm, X):

    W1=lm.model.get_weights()[0]
    b1=lm.model.get_weights()[1]
    W2=lm.model.get_weights()[2]
    b2=lm.model.get_weights()[3]


    M = np.shape(X)[0]
    p = np.shape(X)[1]

    beta=np.array([0]*M*(p+1), dtype='float32').reshape(M,p+1)

    beta[:,0]= (np.dot(np.transpose(W2),np.tanh(b1)) + b2)[0] # intercept \beta_0= F_{W,b}(0)
    for i in range(M):

      Z1 = np.tanh(np.dot(np.transpose(W1),np.transpose(X[i,])) + b1) 

      D = np.diag(1-Z1**2)

      for j in range(p):  
          beta[i,j+1]=np.dot(np.transpose(W2),np.dot(D,W1[j]))

    return(beta)

For reference, below is a table that shows the means and standard deviations of the sensitivities as the number of hidden units is increased.  This is shown for one hidden layer with tanh activation.

In [0]:
array_mean = []
array_std = []
for k in range(len(n)):
  
  # with non-linear activation
  def linear_NN1_model_act(l1_reg=0.0):    
      model = Sequential()
      model.add(Dense(n[k], input_dim=2, kernel_initializer='normal', activation='tanh'))
      model.add(Dense(1, kernel_initializer='normal')) 
      model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
      return model
   
  lm = KerasRegressor(build_fn=linear_NN1_model_act, epochs=100, batch_size=10, verbose=0, callbacks=[es])
  lm.fit(X,Y)
  
  beta=sensitivities_t(lm, X)
  # Check that the intercept is close to one and the coefficients are close to one
  array_mean.append(int(n[k]))
  array_mean.extend(np.mean(beta, axis=0).tolist())
  array_std.append(int(n[k]))
  array_std.extend(np.std(beta, axis=0).tolist())
  
pd_mean = pd.DataFrame(np.reshape(array_mean,(len(n),4)),columns=['number_of_neurons','beta0','beta1','beta2'])
pd_std = pd.DataFrame(np.reshape(array_std,(len(n),4)),columns=['number_of_neurons','beta0','beta1','beta2'])
print(pd_mean)
print(pd_std)

Epoch 00057: early stopping
Epoch 00022: early stopping
Epoch 00015: early stopping
Epoch 00021: early stopping
Epoch 00051: early stopping
   number_of_neurons     beta0     beta1     beta2
0               10.0 -0.002274  0.994728  0.967177
1               50.0  0.017609  0.990028  0.987109
2              100.0  0.074269  0.952894  0.963494
3              200.0  0.007517  0.994349  0.969850
4              500.0  0.000408  1.010127  1.002024
   number_of_neurons         beta0     beta1     beta2
0               10.0  1.070991e-07  0.058954  0.057510
1               50.0  2.831221e-07  0.046515  0.045881
2              100.0  3.844407e-06  0.032985  0.033852
3              200.0  7.497021e-08  0.031956  0.031605
4              500.0  9.458969e-09  0.038202  0.038744


# Compare with a FFW Neural Network with one hidden layer (ReLU activated)


For reference, below is a table that shows the means and standard deviations of the sensitivities as the number of hidden units is increased.  This is shown for one hidden layer with ReLU activation.

In [0]:
# Compute the sensitivities
# Assume that the activation function is tanh
def sensitivities_r(lm, X):

    W1=lm.model.get_weights()[0]
    b1=lm.model.get_weights()[1]
    W2=lm.model.get_weights()[2]
    b2=lm.model.get_weights()[3]


    M = np.shape(X)[0]
    p = np.shape(X)[1]

    beta=np.array([0]*M*(p+1), dtype='float32').reshape(M,p+1)

    beta[:,0]= (np.dot(np.transpose(W2),np.maximum(b1,0)) + b2)[0] # intercept \beta_0= F_{W,b}(0)
    for i in range(M):

      Z1 = np.maximum(np.dot(np.transpose(W1),np.transpose(X[i,])) + b1,0) 

      D = np.diag(np.sign(Z1))  

      for j in range(p):  
          beta[i,j+1]=np.dot(np.transpose(W2),np.dot(D,W1[j]))

    return(beta)

In [0]:
array_mean = []
array_std = []
for k in range(len(n)):
  
  # with non-linear activation
  def linear_NN1_model_act(l1_reg=0.0):    
      model = Sequential()
      model.add(Dense(n[k], input_dim=2, kernel_initializer='normal', activation='relu'))
      model.add(Dense(1, kernel_initializer='normal')) 
      model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
      return model
   
  lm = KerasRegressor(build_fn=linear_NN1_model_act, epochs=100, batch_size=10, verbose=0, callbacks=[es])
  lm.fit(X,Y)
  
  beta=sensitivities_r(lm, X)
  # Check that the intercept is close to one and the coefficients are close to one
  array_mean.append(int(n[k]))
  array_mean.extend(np.mean(beta, axis=0).tolist())
  array_std.append(int(n[k]))
  array_std.extend(np.std(beta, axis=0).tolist())
  
pd_mean = pd.DataFrame(np.reshape(array_mean,(len(n),4)),columns=['number_of_neurons','beta0','beta1','beta2'])
pd_std = pd.DataFrame(np.reshape(array_std,(len(n),4)),columns=['number_of_neurons','beta0','beta1','beta2'])
print(pd_mean)
print(pd_std)

Epoch 00050: early stopping
Epoch 00042: early stopping
Epoch 00036: early stopping
Epoch 00023: early stopping
Epoch 00033: early stopping
   number_of_neurons     beta0     beta1     beta2
0               10.0  0.027086  1.001396  1.012632
1               50.0  0.018870  0.945165  0.966890
2              100.0 -0.001559  1.003370  1.010414
3              200.0  0.013321  1.027624  0.989039
4              500.0  0.016759  0.999056  1.116667
   number_of_neurons         beta0     beta1     beta2
0               10.0  6.705523e-07  0.087771  0.099541
1               50.0  6.109476e-07  0.109643  0.132735
2              100.0  7.392217e-08  0.125465  0.133331
3              200.0  3.985938e-07  0.126030  0.139356
4              500.0  5.830266e-07  0.118538  0.159243


# Exercise 5.4: Test the sensitivities function with L hidden layers (tanh activated)

In [3]:
# Compute the sensitivities for a feedforward architecture with L layers
# Assume that the activation function is tanh
def deep_sensitivities(lm, X):
    
    para = lm.model.get_weights()
    W = [] # store weights
    b = [] # store biases
    for index in range(len(para)):
      if index%2 == 0:
        W.append(para[index])
      else: 
        b.append(para[index])
    M = np.shape(X)[0]
    p = np.shape(X)[1]

    beta=np.array([0]*M*(p+1), dtype='float32').reshape(M,p+1)
    
    beta0 = np.dot(np.transpose(W[1]),np.tanh(b[0])) + b[1]
    for i in range(2,L):
      beta0 = np.dot(np.transpose(W[i]),np.tanh(beta0)) + b[i]
    beta[:,0] = beta0[0]
    
    for i in range(M):

        Z1 = np.tanh(np.dot(np.transpose(W[0]),np.transpose(X[i,])) + b[0]) 

        D1 = np.diag(1-Z1**2)

        for j in range(p): 
        
            Z = Z1

            temp = np.dot(np.transpose(W[1]),np.dot(D1,W[0][j]))

            for l in range(1,L-1):

                Z = np.tanh(np.dot(np.transpose(W[l]),np.transpose(Z)) + b[l]) 
                D = np.diag(1-Z**2)
                temp = np.dot(np.transpose(W[l+1]),np.dot(D,temp))
            beta[i,j+1] = temp
                                                                 
    return (beta)

In [38]:
n = 10 # number of units in each hidden layer
L = 4 # number of layers, including one output layer
# with non-linear activation
def linear_NNL_model_act(l1_reg=0.0):    
    model = Sequential()
    model.add(Dense(n, input_dim=2, kernel_initializer='normal', activation='tanh'))
    for l in range(2,L):
      model.add(Dense(n, kernel_initializer='normal', activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal')) 
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    return model
   
lm = KerasRegressor(build_fn=linear_NNL_model_act, epochs=100, batch_size=10, verbose=0, callbacks=[es])
lm.fit(X,Y)

beta = deep_sensitivities(lm, X)
print(beta)

Epoch 00034: early stopping
[[0.07728125 1.0203655  1.0580494 ]
 [0.07728125 1.0396371  1.0720679 ]
 [0.07728125 1.0998473  1.1350926 ]
 ...
 [0.07728125 0.9173199  0.95067984]
 [0.07728125 0.9627277  0.9931845 ]
 [0.07728125 1.109096   1.1446314 ]]


In [39]:
# Check that the intercept is close to one and the coefficients are close to one
print(np.mean(beta, axis=0))
print(np.std(beta, axis=0))

[0.07727869 1.0217295  1.0542846 ]
[2.5629997e-06 1.0636521e-01 1.1001225e-01]


# Exercise 5.5: Compare the sensitivities with fixed hidden units (tanh activated)

In this problem, we fix the total number of hidden units but change the number of hidden layers.  This allows us to make a conclusion about how the mean and standard deviations of the sensitivities behave as the number of hidden layers is increased.  
Through running the code, it is evident that as the number of hidden layers is increased, the approximations of the sensitivities do not improve.  The means do not converge to the true value of the sensitivities, and the standard deviations appear to grow.  Thus, one might tentatively conclude that keeping the total number of hidden units fixed, the higher the number of hidden layers, the least certain the approximation of the sensitivities will be.

In [41]:
total = 200 # fixed total number of hidden units 
layers = [1, 2, 5, 10] # number of hidden layers
array_mean = []
array_std = []
for k in range(len(layers)):
    n = int(total/layers[k]) # number of units in each hidder layer
    L = layers[k]+1 # number of layers, including one output layer
    # with non-linear activation
    def linear_NNL_model_act(l1_reg=0.0):    
        model = Sequential()
        model.add(Dense(n, input_dim=2, kernel_initializer='normal', activation='tanh'))
        for l in range(2,L):
          model.add(Dense(n, kernel_initializer='normal', activation='tanh'))
        model.add(Dense(1, kernel_initializer='normal')) 
        model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
        return model

    lm = KerasRegressor(build_fn=linear_NNL_model_act, epochs=100, batch_size=10, verbose=0, callbacks=[es])
    lm.fit(X,Y)

    beta = deep_sensitivities(lm, X)
    # Check that the intercept is close to one and the coefficients are close to one
    array_mean.append(layers[k])
    array_mean.extend(np.mean(beta, axis=0).tolist())
    array_std.append(layers[k])
    array_std.extend(np.std(beta, axis=0).tolist())
  
pd_mean = pd.DataFrame(np.reshape(array_mean,(len(layers),4)),columns=['number_of_hidden_layers','beta0','beta1','beta2'])
pd_std = pd.DataFrame(np.reshape(array_std,(len(layers),4)),columns=['number_of_hidden_layers','beta0','beta1','beta2'])
print(pd_mean)
print(pd_std)

Epoch 00016: early stopping
Epoch 00022: early stopping
Epoch 00026: early stopping
Epoch 00029: early stopping
   number_of_hidden_layers     beta0     beta1     beta2
0                      1.0  0.026048  0.986498  0.977701
1                      2.0  0.008938  1.067570  1.031970
2                      5.0 -0.052291  1.039532  0.988219
3                     10.0 -0.164735  1.008504  0.989027
   number_of_hidden_layers         beta0     beta1     beta2
0                      1.0  8.233117e-07  0.028760  0.029629
1                      2.0  4.926731e-07  0.081046  0.078865
2                      5.0  5.774097e-07  0.119299  0.113469
3                     10.0  3.844407e-06  0.186785  0.183237
